# Ensamble Bagging workflow and notebook

### Workflow

In [ ]:
from airflow.operators import CompressFileSensor
from cdcol_utils import other_utils
import airflow
from airflow.models import DAG
from airflow.operators import CDColQueryOperator, CDColFromFileOperator, CDColReduceOperator
from airflow.operators.python_operator import PythonOperator
from cdcol_utils import dag_utils, queue_utils, other_utils
from airflow.utils.trigger_rule import TriggerRule

from datetime import timedelta
from pprint import pprint

_params = {'modelos': '/web_storage/downloads/6863', 'minValid': 1, 'normalized': False, 'lat': (2, 3), 'lon': (-74, -73), 'products': [{'name': 'LS7_ETM_LEDAPS', 'bands': ['swir1', 'red', 'nir', 'green', 'blue', 'swir2', 'pixel_qa']}, {'name': 'LS8_OLI_LASRC', 'bands': ['swir1', 'red', 'nir', 'green', 'blue', 'swir2', 'pixel_qa']}], 'time_ranges': [('2020-01-01', '2020-06-30'), ('2020-01-01', '2020-06-30')], 'execID': 'exec_6863', 'elimina_resultados_anteriores': True, 'genera_mosaico': True, 'owner': 'API-REST'}

# definir unidades FNF, y DEM por defecto (una sola banda)
_params['products'].append({'name': 'DEM_Mosaico', 'bands': ['dem']})
_params['products'].append({'name': 'FNF_COL_UTM', 'bands': ['fnf_mask']})

# Definir periodo de tiempo DEM
_params['time_ranges'] = [('2013-01-01','2013-12-31')] + _params['time_ranges']

# sort params products by name
_params['products'].sort(key = lambda d: d['name'])

"""
Templeate modified my Crhsitian Segura
27-oct-2020
"""
"""
_params = {'minValid': 1, 'normalized': False,
    'modelos': '/web_storage/downloads/3625',
    'lat': (10, 11),
    'lon': (-75, -74),
    'bands': ["red", "nir", "swir1", "swir2"],
    #'minValid': 1,
    'products': [{'name': 'LS7_ETM_LEDAPS_MOSAIC', 'bands': ['swir2', 'nir', 'red', 'swir1']},{'name': 'DEM_Mosaico', 'bands': ['dem']},{'name': 'FNF_COL_UTM', 'bands': ['fnf_mask']}],
    'time_ranges': [('2016-01-01', '2016-12-31'),('2013-01-01', '2013-12-31'),('2017-01-01', '2017-12-31')],
    'execID': "ctm_b_01",
    'elimina_resultados_anteriores': True,
    'genera_mosaico': True,
    #'genera_geotiff': True,
    'owner': 'cubo',
    'normalized': False
}
"""
_steps = {
    'mascara': {
        'algorithm': "mascara-landsat",
        'version': '1.0',
        'queue': queue_utils.assign_queue(input_type='multi_temporal', time_range=_params['time_ranges'][2]),
        'params': {'bands': _params['products'][2]['bands']},
    },
    'consulta': {
        'algorithm': "mascara-landsat",
        'version': '2.0',
        'queue': queue_utils.assign_queue(
            input_type='multi_temporal_area',
            time_range=_params['time_ranges'][2],
            lat=_params['lat'], lon=_params['lon']),
        'params': {'bands': _params['products'][2]['bands']},
    },
    'reduccion': {
        'algorithm': "joiner",
        'version': '1.0',
        'queue': 'airflow_xlarge',
        'params': {'bands': _params['products'][2]['bands']},
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
    'medianas': {
        'algorithm': "compuesto-temporal-medianas-indices-wf",
        'version': '1.0',
        'queue': queue_utils.assign_queue(input_type='multi_temporal_unidad',
                                          time_range=_params['time_ranges'][2],
                                          unidades=len(_params['products'])),
        'params': {
            'bands': _params['products'][2]['bands'],
            'minValid': _params['minValid'],
            'normalized': False,
        },
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
    'medianas_dem': {
        'algorithm': "compuesto-temporal-medianas-indices-wf",
        'version': '3.0',
        'queue': queue_utils.assign_queue(input_type='multi_temporal_unidad',
                                          time_range=_params['time_ranges'][2],
                                          unidades=len(_params['products'])),
        'params': {
            'bands': _params['products'][2]['bands'],
            'minValid': _params['minValid'],
            'normalized': False,
        },
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
    'mosaico': {
        'algorithm': "joiner",
        'version': '1.0',
        'queue': 'airflow_xlarge',
        'params': {},
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
    'entrenamiento': {
        'algorithm': "ensemble-training",
        'version': '1.0',
        'queue': queue_utils.assign_queue(
            input_type='multi_area',
            lat=_params['lat'],
            lon=_params['lon']
        ),
        'params': {
            'bands': _params['products'][2]['bands'],
            'train_data_path': _params['modelos']
        },
        'del_prev_result': False,
    },
    'clasificador': {
        'algorithm': "clasificador-ensemble-wf",
        'version': '1.0',
        'queue': 'airflow_xlarge',
        'params': {
            'bands': _params['products'][0]['bands'],
            'modelos': _params['modelos']
        },
        'del_prev_result': _params['elimina_resultados_anteriores'],
    },
     'mascara_fnf': {
        'algorithm': "mascara_fnf",
        'version': '1.0',
        'queue': 'airflow_medium',
        'params': {
            'bands': _params['products'][1]['bands'],
            'modelos': _params['modelos']
        },
        'del_prev_result': True,
    },
    'geotiff': {
        'algorithm': "generate-geotiff",
        'version': '1.0',
        'queue': queue_utils.assign_queue(input_type='multi_area', lat=_params['lat'], lon=_params['lon']),
        'params': {},
        'del_prev_result': False,
    }

}

args = {
    'owner': _params['owner'],
    'start_date': airflow.utils.dates.days_ago(2),
    'execID':_params['execID'],
    'product': _params['products'][2]
}

dag = DAG(
    dag_id=args['execID'], default_args=args,
    schedule_interval=None,
    dagrun_timeout=timedelta(minutes=120))





mascara_0 = dag_utils.queryMapByTile(
    lat=_params['lat'], 
    lon=_params['lon'],
    time_ranges=_params['time_ranges'][2],
    algorithm=_steps['mascara']['algorithm'],
    version=_steps['mascara']['version'],
    product=_params['products'][2],
    params=_steps['mascara']['params'],
    queue=_steps['mascara']['queue'],
    dag=dag,
    task_id="mascara_" + _params['products'][2]['name']
)

if len(_params['products']) > 3:
    mascara_1 = dag_utils.queryMapByTile(
        lat=_params['lat'],
        lon=_params['lon'],
        time_ranges=_params['time_ranges'][2],
        algorithm=_steps['mascara']['algorithm'],
        version=_steps['mascara']['version'],
        product=_params['products'][3],
        params=_steps['mascara']['params'],
        queue=_steps['mascara']['queue'],
        dag=dag,
        task_id="mascara_" + _params['products'][3]['name']
    )

    reduccion_lansat = dag_utils.reduceByTile(
        mascara_0 + mascara_1,
        algorithm=_steps['reduccion']['algorithm'],
        version=_steps['reduccion']['version'],
        queue=_steps['reduccion']['queue'],
        product=_params['products'][2],
        dag=dag, task_id="joined",
        delete_partial_results=_steps['reduccion']['del_prev_result'],
        params=_steps['reduccion']['params'], 
    )
else:
    reduccion_lansat = mascara_0


medianas = dag_utils.IdentityMap(
                                reduccion_lansat,
                                product=_params['products'][2],
                                algorithm=_steps['medianas']['algorithm'],
                                version=_steps['medianas']['version'],
                                task_id="medianas",
                                queue=_steps['medianas']['queue'],
                                dag=dag,
                                delete_partial_results=_steps['medianas']['del_prev_result'],
                                params=_steps['medianas']['params']
)


mascara_dem_mosaic = dag_utils.queryMapByTile(lat=_params['lat'], lon=_params['lon'],
                                       time_ranges=_params['time_ranges'][0],
                                       algorithm=_steps['consulta']['algorithm'],
                                       version=_steps['consulta']['version'],
                                       product=_params['products'][0],
                                       params=_steps['consulta']['params'],
                                       queue=_steps['consulta']['queue'], dag=dag,
                                       task_id="consulta_referencia_" + _params['products'][0]['name'])


mascara_fnf_mosaic = CDColQueryOperator(lat=_params['lat'], lon=_params['lon'],
                                        time_ranges=_params['time_ranges'][1],
                                        algorithm=_steps['consulta']['algorithm'],
                                        version=_steps['consulta']['version'],
                                        product=_params['products'][1],
                                        params=_steps['consulta']['params'],
                                        queue=_steps['consulta']['queue'],
                                        dag=dag,
                                        task_id="consulta_referencia_" + _params['products'][1]['name'])

reduccion = dag_utils.reduceByTile(
                                    medianas + mascara_dem_mosaic,
                                    product=_params['products'][2],
                                    algorithm=_steps['reduccion']['algorithm'],
                                    version=_steps['reduccion']['version'],
                                    queue=_steps['reduccion']['queue'],
                                    dag=dag, task_id="joined_2",
                                    delete_partial_results=_steps['reduccion']['del_prev_result'],
                                    params=_steps['reduccion']['params'],
)




medianas_dem = dag_utils.IdentityMap(
                                reduccion,
                                product=_params['products'][2],
                                algorithm=_steps['medianas_dem']['algorithm'],
                                version=_steps['medianas_dem']['version'],
                                task_id="medianas_dem",
                                queue=_steps['medianas_dem']['queue'],
                                dag=dag,
                                delete_partial_results=_steps['medianas_dem']['del_prev_result'],
                                params=_steps['medianas_dem']['params']
)




mosaico = dag_utils.OneReduce(medianas_dem, task_id="mosaico_consulta",
                              algorithm=_steps['mosaico']['algorithm'],
                              version=_steps['mosaico']['version'],
                              queue=_steps['mosaico']['queue'],
                              delete_partial_results=_steps['mosaico']['del_prev_result'],
                              trigger_rule=TriggerRule.NONE_FAILED, dag=dag)

entrenamiento = dag_utils.IdentityMap(
                                mosaico,
                                algorithm=_steps['entrenamiento']['algorithm'],
                                version=_steps['entrenamiento']['version'],
                                task_id="entrenamiento",
                                queue=_steps['entrenamiento']['queue'],
                                dag=dag,
                                delete_partial_results=_steps['entrenamiento']['del_prev_result'],
                                params=_steps['entrenamiento']['params']
)

clasificador = CDColReduceOperator(
                                task_id="clasificador_generico",
                                algorithm=_steps['clasificador']['algorithm'],
                                version=_steps['clasificador']['version'],
                                queue=_steps['clasificador']['queue'],
                                dag=dag,
                                lat=_params['lat'],
                                lon=_params['lon'],
                                params=_steps['clasificador']['params'],
                                delete_partial_results=_steps['clasificador']['del_prev_result'],

)

mascara_fnf = CDColReduceOperator(algorithm=_steps['mascara_fnf']['algorithm'],
                                       version=_steps['mascara_fnf']['version'],
                                       queue=_steps['mascara_fnf']['queue'],
                                       params=_steps['mascara_fnf']['params'],
                                       delete_partial_results=_steps['mascara_fnf']['del_prev_result'],
                                       dag=dag, task_id="clasificacion_final", to_tiff=True)


entrenamiento>>clasificador
mosaico>>clasificador



clasificador>>mascara_fnf
mascara_fnf_mosaic>>mascara_fnf

sensor_fin_ejecucion = CompressFileSensor(task_id='sensor_fin_ejecucion',poke_interval=60, soft_fail=True,mode='reschedule', queue='util', dag=dag) 
comprimir_resultados = PythonOperator(task_id='comprimir_resultados',provide_context=True,python_callable=other_utils.compress_results,queue='util',op_kwargs={'execID': args['execID']},dag=dag) 
sensor_fin_ejecucion >> comprimir_resultados 

# Mini-algorithms

In [ ]:
import numpy as np
print(product)
print ("Masking " + product['name'])
nodata=-9999
validValues=set()
if product['name']=="LS7_ETM_LEDAPS" or product['name'] == "LS5_TM_LEDAPS":
    validValues=[66,68,130,132]
elif product['name'] == "LS8_OLI_LASRC":
    validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
else:
    raise Exception("Este algoritmo sólo puede enmascarar LS7_ETM_LEDAPS, LS5_TM_LEDAPS o LS8_OLI_LASRC")

cloud_mask = np.isin(xarr0["pixel_qa"].values, validValues)
for band in product['bands']:
    print("entra a enmascarar")
    xarr0[band].values = np.where(np.logical_and(xarr0.data_vars[band] != nodata, cloud_mask), xarr0.data_vars[band], -9999)
output = xarr0

In [ ]:
import xarray as xr
import glob, os,sys

output=None
xarrs=xarrs.values()
for _xarr in xarrs:
    if (output is None):
        output = _xarr
    else:
        output=output.combine_first(_xarr)

#output=xr.auto_combine(list(xarrs))
#output=xr.open_mfdataset("/source_storage/results/compuesto_de_medianas/compuesto-temporal-medianas-wf_1.0/*.nc")
#output=xr.merge(list(xarrs))

In [ ]:
#!/usr/bin/python3
# coding=utf8
import xarray as xr
import numpy as np
print ("Compuesto temporal de medianas para " + product['name'])
print(xarr0)
nodata=-9999
medians = {}
time_axis = list(xarr0.coords.keys()).index('time')
for band in product['bands']:
    print(product['bands'])
    if band != 'pixel_qa':
        datos = xarr0.data_vars[band].values
        allNan = ~np.isnan(datos)

        # Comentada por Aurelio (No soporta multi unidad)
        #if normalized:
        #    m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)

        if normalized:
            m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
            st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)

            # Expand m and st according with the data shape
            # number of coords
            coords_num = len(list(xarr0.coords.keys()))
            l = [ x for x in range(coords_num) if x != time_axis]

            m_new = m
            st_new = st
            for axis in l:
                # If axis is 0  it is equivalent to x[np.newaxis,:]
                # If axis is 1  it is equivalent to x[:,np.newaxis]
                # And so on
                m_new = np.expand_dims(m_new,axis=axis)
                st_new = np.expand_dims(st_new,axis=axis)

            print('Time axis',time_axis)
            print('New axis',l)
            print('m',m.shape)
            print('st',st.shape)
            print('st_new',st_new.shape)
            print('m_new',m_new.shape)
            datos=np.true_divide((datos-m_new), st_new)*np.nanmean(st)+np.nanmean(m)

        medians[band] = np.nanmedian(datos, time_axis)
        medians[band][np.sum(allNan, time_axis) < minValid] = -9999

medians["ndvi"]=np.true_divide(medians["nir"]-medians["red"],medians["nir"]+medians["red"])
medians["nbr"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])
medians["nbr2"]=np.true_divide(medians["swir1"]-medians["swir2"],medians["swir1"]+medians["swir2"])
medians["ndmi"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])
#medians["gndvi"]=np.true_divide(medians["nir"]-medians["green"],medians["nir"]+medians["green"])
medians["rvi"]=np.true_divide(medians["nir"],medians["red"])
medians["nirv"]=(medians["ndvi"] * medians["nir"])
medians["osavi"]=np.true_divide(medians["nir"]-medians["red"],medians["nir"]+medians["red"]+0.16)


print('medians_calculated')
del datos

# > **Asignación de coordenadas**
ncoords=[]
xdims =[]
xcords={}
for x in xarr0.coords:
    if(x!='time'):
        ncoords.append( ( x, xarr0.coords[x]) )
        xdims.append(x)
        xcords[x]=xarr0.coords[x]
variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords) for k, v in medians.items()}
output=xr.Dataset(variables, attrs={'crs':xarr0.crs})
for x in output.coords:
    output.coords[x].attrs["units"]=xarr0.coords[x].units

In [ ]:
#!/usr/bin/python3
# coding=utf8
import xarray as xr
import numpy as np
print ("Compuesto temporal de medianas para " + product['name'])
print(xarr0)
nodata=-9999
medians = {}
time_axis = list(xarr0.coords.keys()).index('time')

print(' lectura de xarr0')
print(xarr0)
print(type(xarr0))
print('asignacion dem')

dem=xarr0["dem"][0].values

print('finalizacion dem')

print('bandas inicio')
print(type(xarr0.data_vars))
print(xarr0.data_vars)

list_bandas=list(xarr0.data_vars)
print('bandas anterior codigo')


for band in list_bandas:
    print('productbands')
    #print(type('product['bands']'))
    #print(product['bands'])
    if band != 'pixel_qa':
        datos = xarr0.data_vars[band].values
        allNan = ~np.isnan(datos)

        # Comentada por Aurelio (No soporta multi unidad)
        #if normalized:
        #    m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)
        #    datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)

        if normalized:
            m=np.nanmean(datos.reshape((datos.shape[time_axis],-1)), axis=1)
            st=np.nanstd(datos.reshape((datos.shape[time_axis],-1)), axis=1)

            # Expand m and st according with the data shape
            # number of coords
            coords_num = len(list(xarr0.coords.keys()))
            l = [ x for x in range(coords_num) if x != time_axis]

            m_new = m
            st_new = st
            for axis in l:
                # If axis is 0  it is equivalent to x[np.newaxis,:]
                # If axis is 1  it is equivalent to x[:,np.newaxis]
                # And so on
                m_new = np.expand_dims(m_new,axis=axis)
                st_new = np.expand_dims(st_new,axis=axis)

            print('Time axis',time_axis)
            print('New axis',l)
            print('m',m.shape)
            print('st',st.shape)
            print('st_new',st_new.shape)
            print('m_new',m_new.shape)
            datos=np.true_divide((datos-m_new), st_new)*np.nanmean(st)+np.nanmean(m)

        medians[band] = np.nanmedian(datos, time_axis)
        medians[band][np.sum(allNan, time_axis) < minValid] = -9999

medians["dem"]=dem
medians["ndvi"]=np.true_divide(medians["nir"]-medians["red"],medians["nir"]+medians["red"])
medians["nbr"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])
#Revisar NBR2 ccgs
medians["nbr2"]=np.true_divide(medians["swir1"]-medians["swir2"],medians["swir1"]+medians["swir2"])
medians["ndmi"]=np.true_divide(medians["nir"]-medians["swir1"],medians["nir"]+medians["swir1"])
#medians["gndvi"]=np.true_divide(medians["nir"]-medians["green"],medians["nir"]+medians["green"])
medians["rvi"]=np.true_divide(medians["nir"],medians["red"])
medians["nirv"]=(medians["ndvi"] * medians["nir"])
#medians["osavi"]=np.true_divide(medians["nir"]-medians["red"],medians["nir"]+medians["red"]+0.16)


print('medians_calculated')
del datos

# > **Asignación de coordenadas**
ncoords=[]
xdims =[]
xcords={}
for x in xarr0.coords:
    if(x!='time'):
        ncoords.append( ( x, xarr0.coords[x]) )
        xdims.append(x)
        xcords[x]=xarr0.coords[x]
variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords) for k, v in medians.items()}
output=xr.Dataset(variables, attrs={'crs':xarr0.crs})
for x in output.coords:
    output.coords[x].attrs["units"]=xarr0.coords[x].units

In [ ]:
import os,posixpath
import re
import xarray as xr
import numpy as np
import gdal
import zipfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm 
from sklearn.svm import SVC

#parametros:
#xarr0: Mosaico del compuesto de medianas
#bands: Las bandas a utilizar
#train_data_path: Ubicación de los shape files .shp

def enmascarar_entrenamiento(vector_data_path, cols, rows, geo_transform, projection, target_value=1):
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds

def rasterizar_entrenamiento(file_paths, rows, cols, geo_transform, projection):
    labeled_pixels = np.zeros((rows, cols))
    for i, path in enumerate(file_paths):
        label = i+1
        print  ("label")
        print (label)
        ds = enmascarar_entrenamiento(path, cols, rows, geo_transform, projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        print  ("labeled_pixels")
        print (labeled_pixels)
        #ds = None
    return labeled_pixels

# The trainning data must be in a zip folder.
train_zip_file_name  = [file_name for file_name in os.listdir(train_data_path) if file_name.endswith('.zip')][0]
train_zip_file_path = os.path.join(train_data_path,train_zip_file_name)
train_folder_path = train_zip_file_path.replace('.zip','')

print('train_zip_file_path',train_zip_file_path)
print('train_folder_path',train_folder_path)
print('tipo_train_path',type(train_folder_path))

zip_file = zipfile.ZipFile(train_zip_file_path)
zip_file.extractall(train_data_path)
zip_file.close()


files = [f for f in os.listdir(train_folder_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]

shapefiles = [os.path.join(train_folder_path, f) for f in files if f.endswith('.shp')]
print('tipo shapefiles')
print(type(shapefiles))
print(shapefiles)


rows, cols = xarr0[product['bands'][0]].shape

print('rows',rows)
print('cols',cols)

_coords=xarr0.coords

print('bandas xarr0',list(xarr0.data_vars))
lista=list(xarr0.data_vars)

#(originX, pixelWidth, 0, originY, 0, pixelHeight)
geo_transform=(_coords["longitude"].values[0], 0.000269995,0, _coords["latitude"].values[0],0,-0.000271302)
proj = xarr0.crs.crs_wkt

#print('shapefile_docs',shapefiles)

labeled_pixels = rasterizar_entrenamiento(shapefiles, rows, cols, geo_transform, proj)

print('data source')

is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]

# Preprocesar:
#nmed=None
#nan_mask=None
#xarrs=list(xarr0.values())
#print(type(xarrs))
#medians1 = xarrs[0]

#print("medianas")
#print(type(medians1))
print("medianas",xarr0)
print("fin consulta mediana")
#print('medians1.datavars',medians1.data_vars.keys())


bands_data=[]


#bands2=list(xarr0.data_vars.keys())


for band in lista:
    #print('bands',product['bands'])
    # pixel_qa is removed from xarr0 by Compuesto Temporal de Medianas
    if band != 'pixel_qa':
        bands_data.append(xarr0[band])
bands_data = np.dstack(bands_data)
training_samples = bands_data[is_train]
print('training_samples')
print(training_samples.shape)

rows, cols, n_bands = bands_data.shape

np.isfinite(training_samples)
_msk=np.sum(np.isfinite(training_samples),1)>1
training_samples= training_samples[_msk,:]
training_labels=training_labels[_msk]

#mascara valores nan por valor no data
mask_nan=np.isnan(training_samples)
training_samples[mask_nan]=-9999
print('training_samples')
print(training_samples)

print('training_labels')
print(training_labels)


print('training_labels')
print(training_labels.shape)

from sklearn.ensemble import ExtraTreesClassifier
#%%time
rf = RandomForestClassifier(n_jobs=-1, n_estimators=500, verbose=1)
dtree=tree.DecisionTreeClassifier(criterion='gini')
svml=SVC(C=1.0,  class_weight='balanced',decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
           max_iter=-1, probability=False, random_state=None, shrinking=True,tol=0.001, verbose=False)
#knn = KNeighborsClassifier(algorithm='brute',n_neighbors=3,metric='mahalanobis')
nn = MLPClassifier(alpha=0.0001,  hidden_layer_sizes=(500,),random_state=None,max_iter=500,activation = 'logistic',solver='adam')
grad_boost=GradientBoostingClassifier(n_estimators=500,learning_rate=1)
extrat = ExtraTreesClassifier(n_estimators=50, max_depth=None,class_weight='balanced')

clf_array=[rf]#extrat,dtree,nn,svml,grad_boost,rf]#svml,nn,grad_boost,extrat,dtree

for clf in clf_array:
    vanilla_scores = cross_val_score(clf, training_samples, training_labels, cv=2, n_jobs=-1)
    bagging_clf = BaggingClassifier(clf)
    bagging_scores = cross_val_score(bagging_clf,training_samples, training_labels, cv=2, 
       n_jobs=-1)

    print ("Mean of: {1:.3f}, std: (+/-) {2:.3f}[{0}]"  
                       .format(clf.__class__.__name__, 
                       vanilla_scores.mean(), vanilla_scores.std()))
    print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n"
                       .format(clf.__class__.__name__, 
                        bagging_scores.mean(), bagging_scores.std()))



# Split train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_samples, training_labels, test_size=0.3)

print(f'Train X {len(X_train)/len(training_samples)*100:.2f}%')
print(f'Train Y {len(y_train)/len(training_samples)*100:.2f}%')
print(f'Test  X {len(X_test )/len(training_samples)*100:.2f}%')
print(f'Test  Y {len(y_test )/len(training_samples)*100:.2f}%')

print('trainning samples',X_train)
print('trainning labels',y_train)



#bagging_clf.fit(training_samples, training_labels)
bagging_clf.fit(X_train, y_train)


# Calculo de y_pred
print('Estimar y con datos de entrada')
y_pred = bagging_clf.predict(X_test)

# Calculo de matrix de confusion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, precision_score

mconf = confusion_matrix(y_test,y_pred)
# Calculo de kappa score
kappa = cohen_kappa_score(y_test, y_pred)
# Calculo de precision score
prec = precision_score(y_test, y_pred,average = 'weighted')

# Save metrics to file
with open(posixpath.join(folder+'metrics.txt'),'w') as file_metrics:
    print(f'matriz de confusion: {mconf}')
    print(f'kappa score: {kappa}')
    print(f'precision score (weighted): {prec}')
    file_metrics.write('matriz de confusion: \n'+str(mconf))
    file_metrics.write('\nkappa score: '+str(kappa))
    file_metrics.write('\nprecision score (weighted): '+str(prec))




print('bagging mean')
print(bagging_scores.mean())

print('bagging scores')
print(bagging_scores.mean())

print('bagging clf')
print(bagging_clf)


# write shapefiles list
file = open(folder+"shapefiles_list.txt", "w")
file.write("shapefiles list = " + "\n".join(shapefiles))
file.close()


outputxcom=posixpath.join(folder,'modelo_random_forest_2.pkl')
with open(outputxcom, 'wb') as fid:
    joblib.dump(bagging_clf, fid)


#classified = outputxcom.classified
#classified.values = classified.values.astype('float32')

In [ ]:
# In[7]:
import xarray as xr
import numpy as np
from sklearn.externals import joblib
import warnings

# In[21]:

# Preprocesar:
nmed=None
nan_mask=None
xarrs=list(xarrs.values())
print(type(xarrs))
medians1 = xarrs[0]
print(type(medians1))
print("medianas")
bands_data = []

print('medians1.datavars',medians1.data_vars.keys())

#rows, cols = xarr0[product['bands'][0]].shape


bands_data=[]

bands2=list(medians1.data_vars.keys())
print('bands2',bands2)


for band in bands2: 
	if band != 'pixel_qa':
   	 bands_data.append(medians1[band])
bands_data = np.dstack(bands_data)

print('bands_data_test2',bands_data)


rows, cols, n_bands = bands_data.shape

print('rows',rows)
print('cols',cols)

print('n_bands',n_bands)



n_samples = rows*cols
flat_pixels = bands_data.reshape((n_samples, n_bands))
#mascara valores nan por valor no data
mask_nan=np.isnan(flat_pixels)
flat_pixels[mask_nan]=-9999
#result = bagging_clf.predict(flat_pixels)
#classification = result.reshape((rows, cols))
import os

print("modelo")

model = None
for file in other_files:
    if file.endswith(".pkl"):
        model = file
        break
if model is None:
    raise "Debería haber un modelo en la carpeta " + modelos

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    bagging_clf = joblib.load(model)
    print(bagging_clf)

#print(bagging_clf)
n_samples = rows*cols
flat_pixels = bands_data.reshape((n_samples, n_bands))
#mascara valores nan por valor no data
mask_nan=np.isnan(flat_pixels)
flat_pixels[mask_nan]=-9999

print('flat_pixels')
print(type(flat_pixels))
print(flat_pixels.min())
print(flat_pixels.max())
print(flat_pixels)


from numpy import inf
                        
flat_pixels[flat_pixels<0]=0
flat_pixels[flat_pixels==-inf]= 0
flat_pixels[flat_pixels>= 1E308]=0
                                    

print('flat_pixels 3')
print(type(flat_pixels))
print(flat_pixels.min())
print(flat_pixels.max())
print(flat_pixels)                          
#np.isfinite(flat_pixels)

_msk=np.sum(np.isfinite(flat_pixels),1)>1
#flat_pixels= flat_pixels[_msk,:]
flat_pixels=flat_pixels[_msk]


print(flat_pixels.min())
print(flat_pixels.max())                      
#result = bagging_clf.predict(flat_pixels)
#classification = result.reshape((rows, cols))

print("clasificacion final")
result = bagging_clf.predict(flat_pixels)
result = result.reshape((rows, cols))
print("fin funcion de clasificacion")
print(result)

print('tipo result')
print(type(result))

#result = bagging_clf.predict(nmed.T)
#result = result.reshape(sp)

# In[ ]:


# In[24]:

coordenadas = []
dimensiones = []
xcords = {}
for coordenada in xarrs[0].coords:
    if (coordenada != 'time'):
        coordenadas.append((coordenada, xarrs[0].coords[coordenada]))
        dimensiones.append(coordenada)
        xcords[coordenada] = xarrs[0].coords[coordenada]

valores = {"classified": xr.DataArray(result, dims=dimensiones, coords=coordenadas)}
#array = xr.DataArray(result, dims=dimensiones, coords=coordenadas)
#array.astype('float32')
#valores = {"classified": array}

output = xr.Dataset(valores, attrs={'crs': xarrs[0].crs})
for coordenada in output.coords:
    output.coords[coordenada].attrs["units"] = xarrs[0].coords[coordenada].units

classified = output.classified
classified.values = classified.values.astype('float32')

In [ ]:
import sys
import numpy as np
import time
from arrnorm.auxil.auxil import similarity
#import matplotlib.pyplot as plt
from scipy import stats
from arrnorm.auxil.auxil import orthoregress
from operator import itemgetter
from scipy import linalg, stats
import arrnorm.auxil.auxil as auxil
from datetime import datetime
import xarray as xr


# Preprocesar:
nmed=None
nan_mask=None
print('values del directorio')
xarr0=list(xarrs.values())
print(type(xarrs))

#mosaico de LS8

print(xarrs.keys())
inDataset1=[xarrs[k] for k in xarrs.keys() if 'clasificador' in k][0];
#inDataset1.sortby('latitude', ascending=False)
inDataset1 = inDataset1.sortby('latitude', ascending=False)

print('plot_objetivo_inDataset1')
print(inDataset1)

#consulta de mosaico
inDataset2=[xarrs[k] for k in xarrs.keys() if 'consulta_referencia' in k][0];
print('plot_referencia_inDataset2')
print(inDataset2)
print(type(inDataset2))

print(inDataset2["fnf_mask"].values[0]==1)
fnf_mas=np.isin(inDataset2["fnf_mask"].values[0]==1, np.nan)
print('la mascara es tipo')
print(type(fnf_mas))
print(fnf_mas)

fnf_mask_tmp=xr.DataArray(inDataset2["fnf_mask"])

fnf_mask=np.isin(fnf_mask_tmp.values[0]==1, np.nan)
print('la mascara temporal 2 es tipo')
print(type(fnf_mask_tmp))
print(fnf_mask)

print('plot_tipo_objetivo_inDataset1')
print(type(inDataset1))

inDataset12=xr.DataArray(inDataset1["classified"])


#ImgResultadof=np.where(fnf_mask_tmp.values[0]==1, np.nan,inDataset12)
# replace np.nan by 0 to set forest mask to 0
print("fnf masked to 0")
ImgResultadof=np.where(fnf_mask_tmp.values[0]==1, 0,inDataset12)

print(ImgResultadof)
output=ImgResultadof

print("termino funcion")
#print(type(inDataset1[0]))
print("asignacion de dataset")
xarrs[0]=inDataset1




ncoords=[]
xdims =[]
xcords={}
for x in xarrs[0].coords:
    if(x!='time'):
        ncoords.append( ( x, xarrs[0].coords[x]) )
        xdims.append(x)
        xcords[x]=xarrs[0].coords[x]
variables ={"mask": xr.DataArray(output, dims=xdims,coords=ncoords)}
output=xr.Dataset(variables, attrs={'crs':xarrs[0].crs})
for x in output.coords:
    output.coords[x].attrs["units"]=xarrs[0].coords[x].units

print("clasificacion_mask")
#print(output)